In [15]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
content = requests.get('https://www.wssu.edu/admissions/programs/index.html')
soup = BeautifulSoup(content.text)

In [8]:
link_container = list()
template = 'https://www.wssu.edu/admissions/programs/'
for a in soup.find('aside').find_all('a')[3:10]:
    link_container.append(template + a['href'])

In [10]:
link_container = [BeautifulSoup(requests.get(url).text) for url in link_container]

In [34]:
major_lnk = dict()
template = 'https://www.wssu.edu/admissions/programs/'
for link in link_container:
    for a in link.find('article').find_all('a'):
        if re.search(r'\(Major\)|\(Minor\)|\(Major, Minor\)',a.text):
            major = re.search(r'(.*)\s\(',a.text).group(1)
            if re.search(r'http:',a['href']):
                major_lnk[major] = a['href']
            else:
                major_lnk[major] = template + a['href']

In [36]:
for major in major_lnk:
    content = requests.get(major_lnk[major])
    major_lnk[major] = BeautifulSoup(content.text)

In [38]:
scrape = dict.fromkeys(major_lnk.keys())

In [45]:
for major in major_lnk:
    string = ''
    for p in major_lnk[major].find_all('p'):
        string += p.text
    for li in major_lnk[major].find_all('li'):
        string += li.text
    
    scrape[major] = [string]

In [47]:
df = pd.DataFrame.from_dict(data=scrape,orient='index')

In [49]:
df.to_json('Winston-Salem State University.json')